In [1]:
import numpy as np
import pandas as pd
import os
import torch
from datetime import datetime

ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
current_path = os.getcwd()
case_name = 'basecase'
today = datetime.today().strftime("%y%m%d")
print("Today is", today, "\nCurrent path is", current_path)

Today is 230214 
Current path is c:\Joko\Project\Bigdata\randomforest


In [108]:
"""
!!!!!!!!!! 필수 체크 !!!!!!!!!!

RF 모델은 LSTM 결과와 비교를 위해 만들어졌으므로, LSTM에서 만들어진 데이터셋(Train/Valid/Test)을 그대로 가져와서 가공하는 형식으로 코드를 만들었음.

RF 용 데이터로 쪼개야함. LSTM 모델에서 이미 나눠진 데이터를 가지고 와서 내가 원하는 데이터 형태로 바꿀 수 없을까?
Target Features: TVD / MD / (주어진 기간에서) Peak Production rate 까지 걸린 개월 수 / Maximum Peak Production Rate / Cumulative Produciton rate / Re-fracking Information / EUR (= Cumulative production rate ~ 36개월)
TVD, MD를 제외하고 모두 이미 만들어진 파일로 사용 가능. 그러면 TVD / MD 를 Completion에서 매칭해서 가져오는 코드를 작성하자. --> 이러면 나중에 내가 원하는 변수를 따로 가져올 수 있음.

Step 1: 하나의 파일을 불러와서, API number 로 Completion 파일에서 원하는 변수를 찾고 이를 새로운 열로 만드는 것. seq 길이가 안맞는건 어떻게 해결? 안되면 나머진 다 NULL 값 처리
Step 2: 가져온 파일들을 바탕으로 동일 이름의 새로운 Randomforest 용 하나의 dataframe 생성. 즉 Train 용 / Valid용 / Test용 dataset 존재
"""
input_path_fornamelst = current_path + '/data/basecase_from_LSTM/'
namelst = os.listdir(input_path_fornamelst)

if not os.path.exists(current_path + '\\data'):
        os.makedirs(current_path + '\\data')

if not os.path.exists(current_path + '\\data\\' + case_name):
        os.makedirs(current_path + '\\data\\' + case_name)

if not os.path.exists(current_path + '\\data\\' + case_name + '\\case1'):
        os.makedirs(current_path + '\\data\\' + case_name + '\\case1')

input_path_fornamelst_final = current_path + '/data/basecase_from_LSTM/case1'
namelst_final = os.listdir(input_path_fornamelst_final)

In [112]:
#### Control Pannel ####
extrac_lst = ['TVD_FT', 'MD_FT'] # completion data 열 이름과 반드시 동일해야함.
used_period_lst = [6, 12, 24]
maximum_used_period = used_period_lst[-1]

EUR_standard = 36
#### Control Pannel ####

# for dirname, _, filenames in os.walk(current_path + input_path_LSTM): ## 한번에 불러오는 건 담에 하자.
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


source_path = current_path + '/source/completion/'
source_lst = os.listdir(source_path)
source_lst
for source in source_lst:
    source_comp = pd.read_csv(source_path + source)
    df_comp = pd.DataFrame(source_comp)

for directory in namelst_final:

    input_path = current_path + '/data/basecase_from_LSTM/case1/' + directory + '/'
    input_lst = os.listdir(input_path)

    prod_rate_lst = []
    for mon in used_period_lst: ## Peak prod rate 값
        prod_rate_lst = np.append(prod_rate_lst, 'peak_prod_used_' + str(mon) + 'mon')
    for mon in used_period_lst: ## Peak prod rate일 때의 월
        prod_rate_lst = np.append(prod_rate_lst, 'time_to_peak_used_' + str(mon) + 'mon')
    for mon in range(maximum_used_period): ## 월별 생산량
        prod_rate_lst = np.append(prod_rate_lst, 'prod_rate_' + str(mon+1) + 'mon')
    for mon in range(maximum_used_period): ## 주어진 기간 동안 cum 생산량
        prod_rate_lst = np.append(prod_rate_lst, 'cum_rate_' + str(mon) + 'mon')
    prod_rate_lst = np.append(prod_rate_lst, 'EUR') ## EUR 값 = Target = Label

    tmp_lst = np.concatenate((extrac_lst, prod_rate_lst))
    default_lst = ['API', 'shutin', 'refracking']
    column_lst = np.concatenate((default_lst, tmp_lst))

    df = pd.DataFrame(columns=column_lst)

    forsetting_lst = [] ## Data를 하나씩 넣어주기 위해 우선 열 길이에 맞는 nan 값을 생성해서 할당하기 위한 lst
    for _ in range(len(column_lst)):
        forsetting_lst = np.append(forsetting_lst, 'NaN')

    ## 반복문 시작 ##

    for file in input_lst:
        source_train = pd.read_csv(input_path + file)
        df_LSTM = pd.DataFrame(source_train)
        df_LSTM.head()
        df_LSTM['cum_gas_rate'] = 0

        for idx in range(len(df_LSTM)):
            if idx == 0:
                df_LSTM['cum_gas_rate'][idx] = df_LSTM['Gas_rate'][idx]
            else:
                df_LSTM['cum_gas_rate'][idx] = df_LSTM['Gas_rate'][idx] + df_LSTM['cum_gas_rate'][idx-1]

        filename = file.strip('.csv')

        df_tmp = pd.DataFrame(columns=column_lst)
        df_tmp.loc[0] = forsetting_lst
        df_tmp['API'] = filename

        if df_LSTM['ShutinMonths'].sum() == 0:
            df_tmp['shutin'] = 0
        else: df_tmp['shutin'] = 1

        if df_LSTM['Refrac'].sum() == 0:
            df_tmp['refracking'] = 0
        else: df_tmp['refracking'] = 1

        for ext in extrac_lst: # completion source에서 원하는 정보 뽑는 코드 !!! 주의 !!! 여기서 같은 API에 서로 다른 내용이 여러개 있을 수 있는데 그럴 경우 일괄적으로 그냥 맨 위에 값 썼음. 특별한 이유는 X. 다른 방법이 없어서임.
            df_tmp[ext] = df_comp[df_comp['API_UWI'] == filename][ext].values[0]

        for period in used_period_lst:
            df_tmp['peak_prod_used_' + str(period) + 'mon'] = df_LSTM.loc[df_LSTM['Gas_rate'][:period].idxmax()]['Gas_rate']
            df_tmp['time_to_peak_used_' + str(period) + 'mon'] = df_LSTM.loc[df_LSTM['Gas_rate'][:period].idxmax()]['TotalProdMonths']

        for mon in range(maximum_used_period): ## 월별 생산량
            df_tmp['prod_rate_' + str(mon+1) + 'mon'] = df_LSTM['Gas_rate'][mon]
            df_tmp['cum_rate_' + str(mon+1) + 'mon'] = df_LSTM['cum_gas_rate'][mon]

        df_tmp['EUR'] = df_LSTM['cum_gas_rate'][EUR_standard-1]

        df = pd.concat([df, df_tmp])

    df.to_csv(current_path + '/data/' + case_name + '/case1/' + directory + '.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_11120\2938694849.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LSTM['cum_gas_rate'][idx] = df_LSTM['Gas_rate'][idx]
C:\Users\User\AppData\Local\Temp\ipykernel_11120\2938694849.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_LSTM['cum_gas_rate'][idx] = df_LSTM['Gas_rate'][idx]
C:\Users\User\AppData\Local\Temp\ipykernel_11120\2938694849.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [113]:
"""

여기까지가 dataset 구축 단계
이제 데이터를 불러와서, X_train Y_train // X_valid Y_valid 만들어주고 --> 이 때 핵심은 used_period 에 따라 세 개의 모델을 각각 학습하도록 만들기 (데이터셋 구축 자체가 달라짐. 일단은 6개월로 해보자.)
데이터셋 구축했으면, RF 모델 영빈이가 만든거랑 비슷하게 만들어서 돌려보면 됨!
그 다음 Testset 똑같이 불러와서 regression 한 결과만 LSTM이랑 똑같은 형식으로 뽑아주면 끝

"""

"df[df['API'] == '30-015-32997']"

In [155]:
input_path = current_path + '/data/basecase/case1/'
train_source = pd.read_csv(input_path + 'Train.csv')
df_train = pd.DataFrame(train_source)
display(df_train.head())

test_source = pd.read_csv(input_path + 'Test.csv')
df_test = pd.DataFrame(test_source)
display(df_test.head())

,Unnamed: 0,API,shutin,refracking,TVD_FT,MD_FT,peak_prod_used_6mon,peak_prod_used_12mon,peak_prod_used_24mon,time_to_peak_used_6mon,...,cum_rate_16mon,cum_rate_17mon,cum_rate_18mon,cum_rate_19mon,cum_rate_20mon,cum_rate_21mon,cum_rate_22mon,cum_rate_23mon,EUR,cum_rate_24mon
0,0,30-015-20871,1,1,9286.0,11668.0,270.258065,270.258065,270.258065,4.0,...,2587.656989,2722.323656,2900.856989,3032.523656,3298.290323,3485.096774,3660.548387,3828.976959,6959.861854,4050.396313
1,0,30-015-21529,1,0,13544.0,13544.0,2896.652174,2896.652174,2896.652174,1.0,...,23966.570684,24802.796490,25784.451663,26674.032308,27603.832308,28508.735534,29246.102200,30153.521555,39933.533408,30896.618329
2,0,30-015-22627,1,1,13117.0,13117.0,1850.733333,1850.733333,1850.733333,1.0,...,9497.915668,9636.076959,10169.697648,10595.923455,11219.886418,11662.144482,12012.064482,12392.754137,14856.046230,12684.915428
3,0,30-015-23131,1,0,12485.0,12485.0,3533.774194,3533.774194,3533.774194,4.0,...,40840.963274,42723.253597,44528.253597,46309.414887,48132.014887,49831.914887,51668.914887,53320.527790,72574.691461,54958.463274
4,0,30-015-25033,0,0,12222.0,12226.0,941.806452,941.806452,941.806452,1.0,...,13720.692627,14537.241014,15323.111982,16152.433410,16952.981797,17762.048464,18566.854916,19366.154916,28740.965668,20147.864593


,Unnamed: 0,API,shutin,refracking,TVD_FT,MD_FT,peak_prod_used_6mon,peak_prod_used_12mon,peak_prod_used_24mon,time_to_peak_used_6mon,...,cum_rate_16mon,cum_rate_17mon,cum_rate_18mon,cum_rate_19mon,cum_rate_20mon,cum_rate_21mon,cum_rate_22mon,cum_rate_23mon,EUR,cum_rate_24mon
0,0,30-015-31937,1,0,12280.0,12280.0,4728.322581,4851.709677,4851.709677,6.0,...,56637.834985,58600.447888,60409.028533,62109.928533,63673.528533,65183.961866,66508.574770,67501.252189,77498.562656,68614.976327
1,0,30-015-32116,1,0,7996.0,8000.0,2507.096774,2717.107143,2717.107143,5.0,...,32106.264058,32573.464058,33213.364058,33676.622122,34798.722122,35449.947929,36385.214596,38055.111147,56536.814722,40079.207922
2,0,30-015-32181,1,0,7992.0,9850.0,1208.933333,1450.233333,1450.233333,4.0,...,15410.230256,16093.552837,16617.262514,17071.095848,17821.773267,18400.504036,18982.052424,19451.485757,21904.389294,19951.085757
3,0,30-015-32298,1,0,12230.0,12230.0,1747.838710,1747.838710,1747.838710,4.0,...,17079.529721,17744.852301,18467.252301,19211.703914,19878.768431,20507.589859,21130.073730,21740.907063,28322.392760,22271.584483
4,0,30-015-32340,1,0,11757.0,11757.0,631.733333,631.733333,631.733333,2.0,...,6005.925217,6305.699410,6583.232744,6850.748873,7110.082206,7364.082206,7630.275754,7931.535014,11311.857518,8198.825336


In [156]:
########################### Start of Control Pannel #########################
used_period_lst         = [6, 12, 24]  ## 리스트 형태로 들어갈 경우 각 요소별로 모델을 따로 만들어서 결과를 확인함

EUR_standard            = 36 ## 사실 이건 못 바꿈. 이거 바꾸려면 데이터셋 설정부터 다시해야함

use_or_not_shutin       = 1
use_or_not_refracking   = 1
use_or_not_TVD          = 1
use_or_not_MD           = 1

use_or_not_peak_prod    = 1
use_or_not_time_to_peak = 1
use_or_not_prod_rate    = 1
use_or_not_cum_rate     = 0


########################### End of Control Pannel ###########################


input fetatures are ['shutin' 'refracking' 'TVD_FT' 'MD_FT' 'peak_prod_used_6mon'
 'time_to_peak_used_6mon' 'prod_rate_1mon' 'prod_rate_2mon'
 'prod_rate_3mon' 'prod_rate_4mon' 'prod_rate_5mon' 'prod_rate_6mon']

x_train shape is (1894, 12) y_train shape is (1894,)
x_test shape is (407, 12) y_test shape is (407,)


In [165]:
## Train part

from sklearn.ensemble import RandomForestRegressor

for used_period in used_period_lst:
    input_fetures = []
    if use_or_not_shutin == 1:
        input_fetures = np.append(input_fetures, 'shutin')
    if use_or_not_refracking == 1:
        input_fetures = np.append(input_fetures, 'refracking')
    if use_or_not_TVD == 1:
        input_fetures = np.append(input_fetures, 'TVD_FT')
    if use_or_not_MD == 1:
        input_fetures = np.append(input_fetures, 'MD_FT')

    if use_or_not_peak_prod == 1:
        input_fetures = np.append(input_fetures, 'peak_prod_used_' + str(used_period) + 'mon')
    if use_or_not_time_to_peak == 1:
        input_fetures = np.append(input_fetures, 'time_to_peak_used_' + str(used_period) + 'mon')

    if use_or_not_prod_rate == 1:
        for mon in range(used_period):
            input_fetures = np.append(input_fetures, 'prod_rate_' + str(mon+1) + 'mon')
    if use_or_not_cum_rate == 1:
        for mon in range(used_period):
            input_fetures = np.append(input_fetures, 'cum_rate_' + str(mon+1) + 'mon')


    print('input fetatures are', input_fetures)
    x_train = df_train[input_fetures]
    y_train = df_train['EUR']

    x_test = df_test[input_fetures]
    y_test = df_test['EUR']

    print()
    print('x_train shape is', x_train.shape, 'y_train shape is', y_train.shape)
    print('x_test shape is', x_test.shape, 'y_test shape is', y_test.shape)

    RF_model = RandomForestRegressor(n_estimators=100, random_state=0)
    x_train = x_train.fillna(0)
    RF_model.fit(x_train, y_train) ## 일단 NaN 값을 0으로 처리하자. NaN이 있는 데이터를 Drop 할 수도 있지만.

    x_test = x_test.fillna(0)
    y_pred = RF_model.predict(x_test)

    colums_EUR = ['name', 'True', 'Pred']
    df_EUR = pd.DataFrame(columns=colums_EUR)
    df_EUR['name'] = df_test['API'].values
    df_EUR['True'] = y_test
    df_EUR['Pred'] = y_pred
    df_EUR.to_csv(current_path + '/results/' + case_name + '/EUR_scatterplot_' + str(used_period) + '.csv')

input fetatures are ['shutin' 'refracking' 'TVD_FT' 'MD_FT' 'peak_prod_used_6mon'
 'time_to_peak_used_6mon' 'prod_rate_1mon' 'prod_rate_2mon'
 'prod_rate_3mon' 'prod_rate_4mon' 'prod_rate_5mon' 'prod_rate_6mon']

x_train shape is (1894, 12) y_train shape is (1894,)
x_test shape is (407, 12) y_test shape is (407,)
input fetatures are ['shutin' 'refracking' 'TVD_FT' 'MD_FT' 'peak_prod_used_12mon'
 'time_to_peak_used_12mon' 'prod_rate_1mon' 'prod_rate_2mon'
 'prod_rate_3mon' 'prod_rate_4mon' 'prod_rate_5mon' 'prod_rate_6mon'
 'prod_rate_7mon' 'prod_rate_8mon' 'prod_rate_9mon' 'prod_rate_10mon'
 'prod_rate_11mon' 'prod_rate_12mon']

x_train shape is (1894, 18) y_train shape is (1894,)
x_test shape is (407, 18) y_test shape is (407,)
input fetatures are ['shutin' 'refracking' 'TVD_FT' 'MD_FT' 'peak_prod_used_24mon'
 'time_to_peak_used_24mon' 'prod_rate_1mon' 'prod_rate_2mon'
 'prod_rate_3mon' 'prod_rate_4mon' 'prod_rate_5mon' 'prod_rate_6mon'
 'prod_rate_7mon' 'prod_rate_8mon' 'prod_rate_9m

In [ ]:
"""
이부분은 나중을 위한 부분
"""

from sklearn.model_selection import train_test_split
import category_encoders as ce

### categorical variable 사용 시 예시 코드 ordinalEncoder 말고 Onehotcoding 등도 있음 함수는 찾아봐
# encoder = ce.OrdinalEncoder(cols=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])


# X_train = encoder.fit_transform(X_train)

# X_test = encoder.transform(X_test)